In [32]:
from backend.database import engine
from sqlmodel import Session, select, text
from commons.schemas import *

from backend.queries import *

import pandas as pd

db = Session(engine)

In [36]:

def query_per_competition(comp_id: int):
    query = f"""
        SELECT skater.first_name, skater.last_name, category.level, category.age, category.genre, performance.rank, performance.total_segment_score, performance.total_element_score, performance.total_component_score, performance.total_deductions, performance.total_entries, club.abbrev, competition.name
        FROM performance
        JOIN skater ON performance.skater_id = skater.id
        JOIN club ON skater.club_id = club.id
        JOIN category ON performance.category_id = category.id
        JOIN competition ON category.competition_id = competition.id
        WHERE 
            competition.id = {comp_id}
        ORDER BY category.level, category.age, category.genre, performance.rank
    """
    return text(query)


In [37]:

# tcp_competitions = pd.read_sql(query_competitions_from_club("TOUCP"), db.connection())
# print(tcp_competitions)
perf_competitions = pd.read_sql(query_per_competition(1), db.connection())
print(perf_competitions)


2024-04-02 21:54:04,653 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname_1)s
2024-04-02 21:54:04,654 INFO sqlalchemy.engine.Engine [cached since 3928s ago] {'table_name': <sqlalchemy.sql.elements.TextClause object at 0x140d5f080>, 'param_1': 'r', 'param_2': 'p', 'param_3': 'f', 'param_4': 'v', 'param_5': 'm', 'nspname_1': 'pg_catalog'}
2024-04-02 21:54:04,655 INFO sqlalchemy.engine.Engine 
        SELECT skater.first_name, skater.last_name, category.level, category.age, category.genre, performance.rank, performance.total_segment_score, performance.total_element_score, performance.tota

In [ ]:
perf_competitions.groupby("club.abbrev").apply(lambda x: x.sort_values("category.level", ascending=False)).reset_index(drop=True)